# Lab Exercise 1. Scraping Static Websites


This is the warmup task for the first laboratory exercise. It consists of scraping static Websites with BeautifulSoap.

 It should be completed at home and presented at the laboratory.

**Total points: 2**

### Task Description

Scrape the information about the products on the following page:
https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/

For each product scrape:


*   Product title (selector `'.wd-entities-title'`)
*   Product regular price (selector `'.woocommerce-Price-amount'`)
*   Product discount price (if available), same selector as regular price
*   URL to the product page
*   Add to cart button URL

***Help: There are multiple product pages, for each page you need to send a separate request***


Save the results as a DataFrame object

You can add as many code cells as you need.

________________________________________________________________

### Requirements

Import libraries and modules that you are going to use

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Send HTTP request to the target Website

In [3]:
website = requests.get("https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/")

check the response status code

In [4]:
website.status_code

200

### Parse the response content with BeautifulSoap

In [5]:
bs = BeautifulSoup(website.text,"html.parser")

### Extract data from the BeautifulSoap object using any selectors, attribute identifiers, etc.

* Product title (selector '.wd-entities-title')
* Product regular price (selector '.woocommerce-Price-amount')
* Product discount price (if available), same selector as regular price
* URL to the product page
* Add to cart button URL

In [8]:
data = bs.find_all("div",{"class":"product-wrapper"})
title = data[0].find("h3",{"class":"wd-entities-title"}).text
url = data[0].find("h3",{"class":"wd-entities-title"}).find("a")["href"]
butt_url = data[0].find("a",{"class":"add_to_cart_button"})["href"]
prices = data[0].find("span",{"class":"price"}).find_all("span",{"class":"amount"})
reg_price = prices[0].find("bdi").text
disc_price = None
if len(prices) > 1:
    disc_price = prices[1].find("bdi").text


Repeat the extraction process for each page of products

In [9]:
pages_num = len(bs.find("ul",{"class":"page-numbers"}).find_all("li"))
products = []

for page in range(1,pages_num+1):
    website = requests.get(f'https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/{page}/?per_page=36')
    if website.status_code == 200:
        bs = BeautifulSoup(website.text,"html.parser")
        data = bs.find_all("div",{"class":"product-wrapper"})
        for item in data:
            title = item.find("h3",{"class":"wd-entities-title"}).text
            url = item.find("h3",{"class":"wd-entities-title"}).find("a")["href"]
            butt_url = item.find("a",{"class":"add_to_cart_button"})["href"]
            prices = item.find("span",{"class":"price"}).find_all("span",{"class":"amount"})
            reg_price = prices[0].find("bdi").text
            disc_price = None
            if len(prices) > 1:
                disc_price = prices[1].find("bdi").text
            products.append((title,reg_price,disc_price,url,butt_url))

### Create a pandas DataFrame with the scraped products

In [10]:
df = pd.DataFrame(products,columns=["title","regular_price","discount_price","product_url","add_button_url"])

Save the dataframe as `.csv`

In [11]:
df.to_csv("csv/it_products.csv")